In [86]:
from dotenv import load_dotenv
import os
import praw
import re
import requests
import hashlib
import time
import json

# Load environment variables
load_dotenv(override=True)

# Reddit API credentials
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
REDIRECT_URI = os.getenv("REDIRECT_URI")
USER_AGENT = os.getenv("USER_AGENT")
PLAYER_ID = os.getenv("PLAYER_ID")

# Reddit subreddit and keyword to monitor
SUBREDDIT = "whiteoutsurvival"
KEYWORD = "gift code"

# Redemption API endpoint
URL = "https://wos-giftcode-api.centurygame.com/api"


HTTP_HEADER = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "application/json",
}

# Secret key for sign generation
SALT = "tB87#kPtkxqOS2"

RESULT_FILE = "result.json"


In [87]:
# Initialize Reddit API client
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT
)

In [88]:
# Define a function to extract the code from the post text
def extract_code(post_text):
    # Regex to match **Code:** followed by the desired code
    match = re.search(r"\*\*Code:\*\*\s*(\S+)", post_text)
    if match:
        return match.group(1)  # Return the captured code
    return None

In [89]:
def fetch_latest_codes(subreddit_name, keyword):
    """Fetch latest posts containing gift codes."""
    subreddit = reddit.subreddit(subreddit_name)
    codes = []

    for submission in subreddit.search(query= keyword.lower(), time_filter='month'):
        if submission.is_self:  # Check if it's a text post
            code = extract_code(submission.selftext)
            if code:
                codes.append(code)

    return codes

codes = fetch_latest_codes(SUBREDDIT, KEYWORD)

In [90]:
def generate_sign(fid, timestamp, secret_key):
    """
    Generate the sign parameter using the given fid, timestamp, and secret key.
    """
    data = f"fid={fid}&time={timestamp}{secret_key}"
    return hashlib.md5(data.encode()).hexdigest()

In [91]:
def redeem_code(player_id, salt, code):
    """Redeem gift code on Whiteout Survival."""
    
    request_data = {
        "fid": player_id, 
        "time": time.time_ns()
    }

    request_data["sign"] = hashlib.md5(
            (
                "fid=" + request_data["fid"] + "&time=" + str(request_data["time"]) + salt
            ).encode("utf-8")).hexdigest()

    # Login the player
    # It is enough to send the POST request, we don't need to store any cookies/session tokens
    # to authenticate during the next request
    login_request = requests.post(
            URL + "/player", data=request_data, headers=HTTP_HEADER, timeout=30
        )
    
    login_response = login_request.json()

    if login_response["msg"] != "success":
        return print(
            f"Login not possible for player: {player_id}"
        )
        

    # Create the request data that contains the signature and the code
    request_data["cdk"] = code
    request_data["sign"] = hashlib.md5(
        (
            "cdk="
            + request_data["cdk"]
            + "&fid="
            + request_data["fid"]
            + "&time="
            + str(request_data["time"])
            + SALT
        ).encode("utf-8")
    ).hexdigest()

    # Send the gif code redemption request
    redeem_request = requests.post(
        URL + "/gift_code", data=request_data, headers=HTTP_HEADER, timeout=30
    )
    redeem_response = redeem_request.json()

    # In case the gift code is broken, exit straight away
    if redeem_response["err_code"] == 40014:
        print("\nThe gift code doesn't exist!")
    elif redeem_response["err_code"] == 40007:
        print("\nThe gift code is expired!")
    elif redeem_response["err_code"] == 40008:  # ALREADY CLAIMED
        print("Successful")
    elif redeem_response["err_code"] == 20000:  # SUCCESSFULLY CLAIMED
        print("Successful")
    elif redeem_response["err_code"] == 40004:  # TIMEOUT RETRY
        print("Unsuccessful")
    else:
        print("Unsuccessful")
        print("\nError occurred: " + str(redeem_response))


In [92]:
for code in codes:
    print(f"Found code: {code}")
    redeem_code(PLAYER_ID, SALT, code)


Found code: Discord1200K

The gift code is expired!
Found code: seijin2025
Unsuccessful

Error occurred: {'code': 1, 'data': [], 'msg': 'USED.', 'err_code': 40005}
Found code: Xmas2YOU

The gift code is expired!
Found code: howasaba2024
Unsuccessful

Error occurred: {'code': 1, 'data': [], 'msg': 'SAME TYPE EXCHANGE.', 'err_code': 40011}
Found code: HugDay25
Successful
Found code: 2025TPGSWC
Successful
Found code: Happy25

The gift code is expired!
Found code: howasabajp
Unsuccessful

Error occurred: {'code': 1, 'data': [], 'msg': 'SAME TYPE EXCHANGE.', 'err_code': 40011}
Found code: sparkle

The gift code is expired!
